In [1]:
import pickle
with open("../app/models/sentence_similarity_model", "rb") as f:
    model_1 = pickle.load(f)

In [3]:
import pandas as pd
dataset = pd.read_csv("../Data/movie_synopsis.csv")

In [36]:
from sklearn.metrics.pairwise import cosine_distances,euclidean_distances
def give_suggestion(user_input:str, model, k:int=5):
    # on encode l'input de l'user avec le même modèle pour avoir un vecteur
    embed_user = model.encode(user_input)
    stock_similarity = []
    stock_suggest_k = []
    # on calcule les distances cosinus
    for embed,title in total:
        #result = (cosine_distances([embed],[embed_user]),title)
        result = (euclidean_distances([embed],[embed_user]),title)
        # elles sont stockés dans la liste sous forme de tuple contenant la distance cosinus et le titre
        stock_similarity.append(result)
    # on trie (ordre croissant) pour avoir le synopsis le plus proche de l'input de l'usr   
    ma_liste_triee = sorted(stock_similarity, key=lambda x: x[0])
    # afficher les premiers résultats etc - on peut changer cette partie en fonction de ce qu'on veut afficher 
    #(idem pour le return)
    #for i, (x, y) in enumerate(ma_liste_triee[:5]):
        #print(f"Suggestion {i+1}: {y} - distance {x}")
        #stock_suggest_k.append(y)
    return stock_similarity #ou stock_suggest_k ou stock_similarity[:k]

In [26]:
dataset.head()

,id,synopsis,title
0,0,It's Ted the Bellhop's first night on the job....,Four Rooms
1,1,"While racing to a boxing match, Frank, Mike, J...",Judgment Night
2,2,Timo Novotny labels his new project an experim...,Life in Loops (A Megacities RMX)
3,3,Princess Leia is captured and held hostage by ...,Star Wars
4,4,"Nemo, an adventurous young clownfish, is unexp...",Finding Nemo


In [31]:
syn_title = []
for i in dataset.index:
    syn_title.append(tuple((dataset["synopsis"][i],dataset["title"][i])))

total = []
for syn, title in syn_title:
    emb = model_1.encode(syn)
    total.append(tuple((emb,title)))

In [37]:
def eval(model):
    value = 0
    for syn, title in syn_title:
        best = give_suggestion(syn, model)[0]
        if best == title:
            value += 1
    return value/len(syn_title)
         

In [38]:
eval(model_1)

0.0